<a target="_blank" href="https://colab.research.google.com/github/Deanis/MLEngineering_Capstone_Group3/blob/main/notebooks/model_training.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup

In [1]:
import tensorflow as tf
print("GPU Available: ", tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU Available:  True


In [4]:
!pip install pandas transformers google-cloud-storage google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 109.2 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.1
    Uninstalling shapely-2.0.1:
      Successfully uninstalled shapely-2.0.1


In [1]:
from google.colab import auth
auth.authenticate_user()

In [18]:
# Provide your bucket name and file name
bucket_name = 'blank-to-bard'
file_name = 'english-dataset.csv'

# Define your project ID and model name
project_id = 'ml-class-group-3-capstone'
model_name = 'blank-to-bard'


## Fetch the dataset from GCS:

In [3]:
from google.cloud import storage
import pandas as pd
from io import BytesIO

# Initialize a client
storage_client = storage.Client()

# Access the bucket and the file
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(file_name)

# Download the data into a pandas dataframe
data = blob.download_as_text()
data = pd.read_csv(BytesIO(bytes(data, 'utf-8')))

# Check the first few records
print(data.head())


                                           text  label
0  The quick brown fox jumps over the lazy dog.      1
1  Dog lazy the over jumps fox brown quick the.      0
2                       The cat sat on the mat.      1
3                       Cat the on mat the sat.      0
4                  She walks to work every day.      1


## Prepare data, train, and evaluate a model using DistilBERT:

In [5]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Tokenize the data
input_ids = []
attention_masks = []

for text in data['text']:
    inputs = tokenizer.encode_plus(text, add_special_tokens=True, max_length=128, pad_to_max_length=True,
                                   return_attention_mask=True, return_tensors='tf')
    input_ids.append(inputs['input_ids'])
    attention_masks.append(inputs['attention_mask'])

input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)

# Ensure labels are in the same order and format as the inputs
labels = tf.convert_to_tensor(data['label'])

# Convert TensorFlow tensors to numpy arrays before splitting
input_ids = input_ids.numpy()
attention_masks = attention_masks.numpy()
labels = labels.numpy()

# Split the data
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2023, test_size=0.2)

# Convert numpy arrays back to tensors
train_inputs = tf.convert_to_tensor(train_inputs)
validation_inputs = tf.convert_to_tensor(validation_inputs)
train_labels = tf.convert_to_tensor(train_labels)
validation_labels = tf.convert_to_tensor(validation_labels)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit([train_inputs, attention_masks[:len(train_inputs)]], train_labels, batch_size=32, epochs=4, validation_split=0.1)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/4
1079/1079 [==============================] - 98s 65ms/step - loss: 0.0564 - accuracy: 0.9799 - val_loss: 0.0114 - val_accuracy: 0.9977
Epoch 2/4
1079/1079 [==============================] - 56s 52ms/step - loss: 0.0129 - accuracy: 0.9967 - val_loss: 0.0180 - val_accuracy: 0.9963
Epoch 3/4
1079/1079 [==============================] - 56s 52ms/step - loss: 0.0092 - accuracy: 0.9974 - val_loss: 0.0051 - val_accuracy: 0.9982
Epoch 4/4
1079/1079 [==============================] - 56s 52ms/step - loss: 0.0063 - accuracy: 0.9981 - val_loss: 0.0038 - val_accuracy: 0.9984


## Save the trained model and upload it to GCS:

\

In [22]:
import os

# Define GCS bucket
bucket = storage_client.bucket(bucket_name)

# Define model directory
model_dir = './model'

# Walk through all directories and sub-directories
for root, dirs, files in os.walk(model_dir):
    for file in files:
        local_file = os.path.join(root, file)
        remote_file = local_file[len(model_dir):]

        # Ensure the path doesn't start with a slash
        if remote_file.startswith('/'):
            remote_file = remote_file[1:]

        blob = bucket.blob(f'model/{remote_file}')
        blob.upload_from_filename(local_file)


## Deploy the model on Vertex AI:

In [24]:
from google.cloud import aiplatform

aiplatform.init(project=project_id)

# gs://blank-to-bard/model/saved_model/1/saved_model.pb

model = aiplatform.Model.upload(
    display_name=model_name,
    artifact_uri="gs://" + bucket_name + "/model/saved_model/1/",
    serving_container_image_uri='gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest',
)

model.wait()

Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/105876446030/locations/us-central1/models/2824383687989133312/operations/1081997713215586304


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/105876446030/locations/us-central1/models/2824383687989133312/operations/1081997713215586304


Model created. Resource name: projects/105876446030/locations/us-central1/models/2824383687989133312@1


INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/105876446030/locations/us-central1/models/2824383687989133312@1


To use this Model in another session:


INFO:google.cloud.aiplatform.models:To use this Model in another session:


model = aiplatform.Model('projects/105876446030/locations/us-central1/models/2824383687989133312@1')


INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/105876446030/locations/us-central1/models/2824383687989133312@1')


## Create an endpoint to deploy your model:



In [36]:
endpoint = aiplatform.Endpoint.create(
    display_name="blank-to-bard",
)

endpoint.deploy(
    model=model,
    deployed_model_display_name=model_name,
    traffic_percentage=100,
)


Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/105876446030/locations/us-central1/endpoints/8571964173955629056/operations/1804825453408550912


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/105876446030/locations/us-central1/endpoints/8571964173955629056/operations/1804825453408550912


Endpoint created. Resource name: projects/105876446030/locations/us-central1/endpoints/8571964173955629056


...

INFO:google.cloud.aiplatform.models:Using default machine_type: n1-standard-2


Deploy Endpoint model backing LRO: projects/105876446030/locations/us-central1/endpoints/8571964173955629056/operations/2746077775528984576


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/105876446030/locations/us-central1/endpoints/8571964173955629056/operations/2746077775528984576


Endpoint model deployed. Resource name: projects/105876446030/locations/us-central1/endpoints/8571964173955629056


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/105876446030/locations/us-central1/endpoints/8571964173955629056


## sample request

In [49]:
!gcloud auth print-access-token

ya29.a0AbVbY6ODSjVOmkZoputAIDSil3w_dbIcs9RbDOoZ4Y1h2fveQN5Mny8hqlNaHXZ7AlJ58oGkoumkV-gPy2LqgJ3X5-ZjBATvdAYaooTB9VujkZUc8AXzCTKsZZXMbIXH_hgPnJA8S87XriBvam3Lw-NvJPqGaCgYKAZcSARISFQFWKvPlUEKQ7Iip_fIpfZ2Xm96HSA0163


In [52]:
import requests
import json

access_token="ya29.a0AbVbY6ODSjVOmkZoputAIDSil3w_dbIcs9RbDOoZ4Y1h2fveQN5Mny8hqlNaHXZ7AlJ58oGkoumkV-gPy2LqgJ3X5-ZjBATvdAYaooTB9VujkZUc8AXzCTKsZZXMbIXH_hgPnJA8S87XriBvam3Lw-NvJPqGaCgYKAZcSARISFQFWKvPlUEKQ7Iip_fIpfZ2Xm96HSA0163"

endpoint_id = "55657278598021120"

# text = "Me no speak"
text = "To be or not to be that is the question."

# Tokenize the text
inputs = tokenizer(text, truncation=True, padding=True, return_tensors="tf")
# Prepare a dictionary matching the model's expected input format
instance = {key: value.numpy().tolist()[0] for key, value in inputs.items()}

input_data = {
    "instances": [instance]
}

# Convert input data to JSON string
input_data_json = json.dumps(input_data)

# Define the API endpoint URL
api_url = f"https://us-central1-aiplatform.googleapis.com/v1/projects/{project_id}/locations/us-central1/endpoints/{endpoint_id}:predict"

# Set the request headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Send the POST request
response = requests.post(api_url, headers=headers, data=input_data_json)

# Parse the response
if response.status_code == 200:
    prediction = response.json()
    print("Prediction:", prediction)
else:
    print("Prediction request failed with status code:", response.status_code)


Prediction: {'predictions': [[-4.57415724, 4.82015753]], 'deployedModelId': '7583272323143172096', 'model': 'projects/105876446030/locations/us-central1/models/2824383687989133312', 'modelDisplayName': 'blank-to-bard', 'modelVersionId': '1'}
